In [5]:
!pip install feedparser
import feedparser
import csv
import pandas as pd
import re

Укажите словарь с RSS-лентой, обратите внимание, что не все RSS-ленты имеют доступный для парсинга формат, проверяйте
<br/>Укажите два вектора по которым будет производить поиск, символ '|' означает "или", если нужен только один вектор, в конце другого добавьте '|' - это будет означать, что вектор ищет все слова. Векторы не чувствительны к регистрам
<br/>Укажите два пути для файлов, для всех новостей и для определенных новостей

In [6]:
newsurls = {'Kommersant': 'https://www.kommersant.ru/RSS/news.xml',
'Lenta.ru': 'https://lenta.ru/rss/',
'Vesti': 'https://www.vesti.ru/vesti.rss'} #пример словаря RSS-лент 
                                           #русскоязычных источников

f_all_news = 'allnews23march.csv'
f_certain_news = 'certainnews23march.csv'

vector1 = 'ДолЛАР|РубЛ|ЕвРО' #пример таргетов
vector2 = 'ЦБ|СбЕРбАНК|курс'

In [7]:
def parseRSS( rss_url ): #функция получает линк на рсс ленту, возвращает распаршенную ленту с помощью feedpaeser
    return feedparser.parse( rss_url )  
    
def getHeadlines( rss_url ): #функция для получения заголовков новости
    headlines = []
    feed = parseRSS( rss_url )
    for newsitem in feed['items']:
        headlines.append(newsitem['title'])
    return headlines

def getDescriptions( rss_url ): #функция для получения описания новости
    descriptions = []
    feed = parseRSS( rss_url )
    for newsitem in feed['items']:
        descriptions.append(newsitem['description'])
    return descriptions

def getLinks( rss_url ): #функция для получения ссылки на источник новости
    links = []
    feed = parseRSS( rss_url )
    for newsitem in feed['items']:
        links.append(newsitem['link'])
    return links

def getDates( rss_url ): #функция для получения даты публикации новости
    dates = []
    feed = parseRSS( rss_url )
    for newsitem in feed['items']:
        dates.append(newsitem['published'])
    return dates

In [8]:
allheadlines = []
alldescriptions = []
alllinks = []
alldates = []
# Прогоняем нашии URL и добавляем их в наши пустые списки
for key,url in newsurls.items():
    allheadlines.extend( getHeadlines( url ) )
    
for key,url in newsurls.items():
    alldescriptions.extend( getDescriptions( url ) )
    
for key,url in newsurls.items():
    alllinks.extend( getLinks( url ) )
    
for key,url in newsurls.items():
    alldates.extend( getDates( url ) )

In [9]:
def write_all_news(all_news_filepath): #функция для записи всех новостей в .csv, возвращает нам этот датасет
    header = ['Title','Description','Links','Publication Date'] 

    with open(all_news_filepath, 'w', encoding='utf-8-sig') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')

        writer.writerow(i for i in header) 

        for a,b,c,d  in zip(allheadlines,alldescriptions,
                            alllinks, alldates):
            writer.writerow((a,b,c,d))


        df = pd.read_csv(all_news_filepath)
            
    return df

def looking_for_certain_news(all_news_filepath, certain_news_filepath, target1, target2): #функция для поиска, а затем записи
                                                                                         #определенных новостей по таргета,
                                                                                        #затем возвращает этот датасет
    df = pd.read_csv(all_news_filepath)
    
    result = df.apply(lambda x: x.str.contains(target1, na=False,
                                    flags = re.IGNORECASE, regex=True)).any(axis=1)
    result2 = df.apply(lambda x: x.str.contains(target2, na=False,
                                    flags = re.IGNORECASE, regex=True)).any(axis=1)
    new_df = df[result&result2]
        
    new_df.to_csv(certain_news_filepath
                     ,sep = '\t', encoding='utf-8-sig')
        
    return new_df

In [10]:
write_all_news(f_all_news) #все новости

,Title,Description,Links,Publication Date
0,Минпромторг сообщил о поэтапной остановке заво...,Производство лопастей на заводе «Вестас Мэньюф...,https://www.kommersant.ru/doc/5605655,"Fri, 07 Oct 2022 20:02:00 +0300"
1,Суд лишил просветительский центр «Мемориал» офиса,Тверской суд Москвы признал незаконной передач...,https://www.kommersant.ru/doc/5605676,"Fri, 07 Oct 2022 19:36:00 +0300"
2,"Безработица в США упала до 3,5%",По данным американского Федерального бюро стат...,https://www.kommersant.ru/doc/5605668,"Fri, 07 Oct 2022 19:28:00 +0300"
3,Песков отрицает раскол в окружении Путина,Пресс-секретарь президента России Дмитрий Песк...,https://www.kommersant.ru/doc/5605665,"Fri, 07 Oct 2022 19:17:00 +0300"
4,"В России в сентябре зафиксировали инфляцию 0,05%",В сентябре 2022 индекс потребительских цен сос...,https://www.kommersant.ru/doc/5605666,"Fri, 07 Oct 2022 19:08:00 +0300"
...,...,...,...,...
363,На Кубу прибыли первые российские чартеры за п...,Два самолета с российскими туристами прибыли н...,https://www.kommersant.ru/doc/5594819,"Wed, 05 Oct 2022 04:37:00 +0300"
364,CNN: США отговаривают страны ОПЕК+ голосовать ...,Белый дом отговаривает страны Персидского зали...,https://www.kommersant.ru/doc/5594818,"Wed, 05 Oct 2022 04:07:00 +0300"
365,Борис Джонсон стал почетным доктором Львовског...,Львовский национальный университет им. Ивана Ф...,https://www.kommersant.ru/doc/5594817,"Wed, 05 Oct 2022 03:37:00 +0300"
366,"В Пентагоне считают, что Украина может атакова...","Представитель Пентагона Лора Купер заявила, чт...",https://www.kommersant.ru/doc/5594816,"Wed, 05 Oct 2022 03:11:00 +0300"


In [11]:
looking_for_certain_news(f_all_news, f_certain_news, vector1, vector2) #новости по вектору

,Title,Description,Links,Publication Date
11,Индекс РТС опустился ниже 1000 пунктов впервые...,Долларовый индекс РТС на Мосбирже снижался до ...,https://www.kommersant.ru/doc/5605644,"Fri, 07 Oct 2022 18:00:00 +0300"
76,Доллар подорожал до 61 рубля,После открытия торгов на Мосбирже доллар США п...,https://www.kommersant.ru/doc/5596123,"Fri, 07 Oct 2022 08:35:00 +0300"
157,МИД поставил под «большой вопрос» энергетическ...,Из-за инцидентов на «Северных потоках» и запад...,https://www.kommersant.ru/doc/5595714,"Thu, 06 Oct 2022 15:02:00 +0300"
190,Объем частных инвестиций в российские акции в ...,В сентябре 2022 года частные инвесторы вложили...,https://www.kommersant.ru/doc/5595593,"Thu, 06 Oct 2022 11:49:00 +0300"
235,Московский зоопарк собирается купить дружелюбн...,Московский зоопарк намерен купить моржа со спо...,https://www.kommersant.ru/doc/5595414,"Thu, 06 Oct 2022 00:52:00 +0300"
254,Новак: объем добычи нефти составит 530 млн тон...,"Вице-премьер РФ Александр Новак заявил, что бю...",https://www.kommersant.ru/doc/5595278,"Wed, 05 Oct 2022 19:44:00 +0300"
